# Interaction avec les campagnes <a class="anchor" id="top"></a>

Dans ce bloc-notes, vous allez déployer et interagir avec les campagnes dans Amazon Personalize.

1. [Introduction](#intro)
1. [Créer des campagnes](#create)
1. [Interagir avec les campagnes](#interact)
1. [Recommandations par lots](#batch)
1. [Récapitulatif](#wrapup)

## Introduction <a class="anchor" id="intro"></a>
[Retour au début](#top)

À ce stade, vous devriez avoir plusieurs solutions et au moins une version de solution pour chacune d'elles. Une fois qu'une version de la solution est créée, il est possible d'obtenir des recommandations de cette dernière et de s'imprégner de son comportement général.

Ce bloc-notes déploie tout d'abord chacune des versions de la solution du bloc-notes précédent dans des campagnes individuelles. Une fois qu'elles sont actives, des ressources sont prévues pour interroger les recommandations, ainsi que des fonctions d'aide pour transformer le résultat en données plus lisibles par l'homme. 

Comme vous avec votre client sur Amazon Personalize, vous pouvez modifier les fonctions d'aide pour les adapter à la structure de leurs fichiers d'entrée de données afin que le rendu supplémentaire fonctionne.

Pour commencer, nous devons une fois de plus importer des bibliothèques, charger les valeurs des bloc-notes précédents et charger le kit SDK.

In [ ]:
import time
from time import sleep
import json
from datetime import datetime
import uuid
import random

import boto3
import pandas as pd

In [ ]:
%store -r

In [ ]:
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

# Establish a connection to Personalize's event streaming
personalize_events = boto3.client(service_name='personalize-events')

## Interagir avec les campagnes <a class="anchor" id="interact"></a>
[Retour au début](#top)

Maintenant que toutes les campagnes sont déployées et actives, nous pouvons obtenir des recommandations via un appel API. Chacune des campagnes est basée sur une recette différente, qui se comporte de manière légèrement différente, car elle répond à des cas d'utilisation différents. Nous aborderons chaque campagne dans un ordre différent de celui utilisé dans les bloc-notes précédents, afin de traiter les éventuelles complexités par ordre croissant (c'est-à-dire le plus simple en premier).

Tout d'abord, créons une fonction de prise en charge pour faciliter la compréhension des résultats renvoyés par une campagne Personalize. Personalize renvoie uniquement un `item_id`. Cette méthode est très pratique pour conserver des données compactes, mais elle signifie que vous devez interroger une base de données ou une table de consultation pour obtenir un résultat lisible par l'homme pour les bloc-notes. Nous allons créer une fonction d'aide pour renvoyer un résultat lisible par l'homme à partir du jeu de données LastFM.

Commencez par charger le jeu de données que nous pouvons utiliser pour notre table de consultation.

In [ ]:
# Create a dataframe for the items by reading in the correct source CSV
items_df = pd.read_csv(dataset_dir + '/movies.csv', sep=',', usecols=[0,1], encoding='latin-1', dtype={'movieId': "object", 'title': "str"},index_col=0)

# Render some sample data
items_df.head(5)

En définissant la colonne ID comme colonne d'index, il est aisé de trouver un artiste en recherchant simplement l'ID.

In [ ]:
movie_id_example = 589
title = items_df.loc[movie_id_example]['title']
print(title)

Cette méthode est acceptable. Cependant, il serait fastidieux de la répéter partout dans notre code. C'est pourquoi la fonction ci-dessous va la supprimer.

In [ ]:
def get_movie_by_id(movie_id, movie_df=items_df):
    """
    This takes in an artist_id from Personalize so it will be a string,
    converts it to an int, and then does a lookup in a default or specified
    dataframe.
    
    A really broad try/except clause was added in case anything goes wrong.
    
    Feel free to add more debugging or filtering here to improve results if
    you hit an error.
    """
    try:
        return movie_df.loc[int(movie_id)]['title']
    except:
        return "Error obtaining title"

Testons à présent quelques valeurs simples pour vérifier notre système de détection d'erreurs.

In [ ]:
# A known good id (The Princess Bride)
print(get_movie_by_id(movie_id="1197"))
# A bad type of value
print(get_movie_by_id(movie_id="987.9393939"))
# Really bad values
print(get_movie_by_id(movie_id="Steve"))

Super ! Nous disposons désormais d'un moyen de présentation des résultats. 

### SIMS

Il suffit de saisir un article pour que SIMS fournisse des articles avec lesquels les utilisateurs interagissent comme ils le font avec l'article saisi. Dans ce cas particulier, l'article est un film. 

Les cellules ci-dessous prennent en charge l'obtention des recommandations à partir de SIMS et la présentation des résultats. Voyons quelles sont les recommandations pour le premier article que nous avons examiné plus tôt dans ce bloc-notes (Terminator 2 : Le Jugement dernier).

In [ ]:
get_recommendations_response = personalize_runtime.get_recommendations(
    campaignArn = sims_campaign_arn,
    itemId = str(589),
)

In [ ]:
item_list = get_recommendations_response['itemList']
for item in item_list:
    print(get_movie_by_id(movie_id=item['itemId']))

Félicitations ! Ceci est votre première liste de recommandations. Cette liste est parfaite. Toutefois, il serait préférable de voir les recommandations de notre sélection d'artistes présentées dans un dans un tableau de données bien structuré. Nous allons à nouveau créer une fonction d'aide pour y parvenir.

In [ ]:
# Update DF rendering
pd.set_option('display.max_rows', 30)

def get_new_recommendations_df(recommendations_df, movie_ID):
    # Get the movie name
    movie_name = get_movie_by_id(movie_ID)
    # Get the recommendations
    get_recommendations_response = personalize_runtime.get_recommendations(
        campaignArn = sims_campaign_arn,
        itemId = str(movie_ID),
    )
    # Build a new dataframe of recommendations
    item_list = get_recommendations_response['itemList']
    recommendation_list = []
    for item in item_list:
        movie = get_movie_by_id(item['itemId'])
        recommendation_list.append(movie)
    new_rec_DF = pd.DataFrame(recommendation_list, columns = [movie_name])
    # Add this dataframe to the old one
    recommendations_df = pd.concat([recommendations_df, new_rec_DF], axis=1)
    return recommendations_df

Testons à présent la fonction d'aide avec plusieurs films différents. Testons notre campagne SIMS à partir de quelques données de notre jeu de données. Sélectionnez 5 films aléatoires dans notre tableau de données.

Remarque : Nous présenterons des titres similaires. Vous pouvez donc relancer la démonstration jusqu'à ce que vous reconnaissiez certains des films listés.

In [ ]:
samples = items_df.sample(5)
samples

In [ ]:
sims_recommendations_df = pd.DataFrame()
movies = samples.index.tolist()

for movie in movies:
    sims_recommendations_df = get_new_recommendations_df(sims_recommendations_df, movie)

sims_recommendations_df

Vous remarquerez peut-être une ressemblance avec plusieurs articles, mais pas tous (la probabilité est plus grande avec un nombre réduit d'interactions, notamment avec le petit jeu de données movielens). Cela prouve que vous ne devez pas vous baser uniquement sur les métriques d'évaluation pour évaluer la version de votre solution. Que pouvez-vous donc faire pour améliorer les résultats en pareil cas ?

Le moment est tout indiqué pour envisager les hyperparamètres des recettes Personalize. La recette SIMS comporte un hyperparamètre `popularity_discount_factor` (voir la [documentation](https://docs.aws.amazon.com/personalize/latest/dg/native-recipe-sims.html)). L'utilisation de cet hyperparamètre vous permet de contrôler la nuance constatée dans les résultats. Ce paramètre et son comportement sont spécifiques à chaque jeu de données que vous rencontrez et dépendent des objectifs de l'entreprise. Vous pouvez répéter la valeur de cet hyperparamètre jusqu'à ce que vous soyez satisfait des résultats. Vous pouvez aussi commencer par utiliser la fonction d'optimisation des hyperparamètres (HPO) de Personalize. Pour en savoir plus sur les hyperparamètres et le paramétrage de HPO, reportez-vous à la [documentation](https://docs.aws.amazon.com/personalize/latest/dg/customizing-solution-config-hpo.html).

### Personnalisation de l'utilisateur

HRNN est l'un des algorithmes les plus avancés fournis par Amazon Personalize. Il prend en charge la personnalisation des articles pour un utilisateur spécifique sur la base de son comportement passé et peut prendre en compte les événements en temps réel afin de modifier les recommandations pour un utilisateur sans nouvel entraînement. 

Le principe HRNN étant de disposer d'un échantillon d'utilisateurs, chargeons les données dont nous avons besoin pour cette opération et sélectionnons trois utilisateurs au hasard. Puisque MovieLens n'inclut pas de données sur les utilisateurs, nous allons sélectionner trois nombres aléatoires dans la gamme des identifiants d'utilisateurs du jeu de données.

In [ ]:
if not USE_FULL_MOVIELENS:
    users = random.sample(range(1, 600), 3)
else:
    users = random.sample(range(1, 162000), 3)
users

Nous présentons ci-après les recommandations pour nos trois utilisateurs aléatoires. Nous explorerons ensuite les interactions en temps réel avant de passer au classement personnalisé.

Nous créons à nouveau une fonction d'aide pour présenter les résultats dans un tableau de données bien structuré.

#### Résultats des appels d'API

In [ ]:
# Update DF rendering
pd.set_option('display.max_rows', 30)

def get_new_recommendations_df_users(recommendations_df, user_id):
    # Get the movie name
    #movie_name = get_movie_by_id(artist_ID)
    # Get the recommendations
    get_recommendations_response = personalize_runtime.get_recommendations(
        campaignArn = userpersonalization_campaign_arn,
        userId = str(user_id),
    )
    # Build a new dataframe of recommendations
    item_list = get_recommendations_response['itemList']
    recommendation_list = []
    for item in item_list:
        movie = get_movie_by_id(item['itemId'])
        recommendation_list.append(movie)
    #print(recommendation_list)
    new_rec_DF = pd.DataFrame(recommendation_list, columns = [user_id])
    # Add this dataframe to the old one
    recommendations_df = pd.concat([recommendations_df, new_rec_DF], axis=1)
    return recommendations_df

In [ ]:
recommendations_df_users = pd.DataFrame()
#users = users_df.sample(3).index.tolist()

for user in users:
    recommendations_df_users = get_new_recommendations_df_users(recommendations_df_users, user)

recommendations_df_users

Ici, nous voyons clairement que les recommandations sont différentes pour chaque utilisateur. Si vous aviez besoin d'un cache pour ces résultats, vous pourriez commencer par exécuter les appels d'API par tous vos utilisateurs et stocker les résultats. Vous pourriez aussi utiliser une exportation par lot. Cette approche qui sera abordée plus tard dans ce bloc-notes.

Appliquons à présent des filtres d'articles pour accéder aux recommandations pour l'un de ces utilisateurs au sein d'un genre.


In [ ]:
def get_new_recommendations_df_by_filter(recommendations_df, user_id, filter_arn):
    # Get the movie name
    #movie_name = get_movie_by_id(artist_ID)
    # Get the recommendations
    get_recommendations_response = personalize_runtime.get_recommendations(
        campaignArn = userpersonalization_campaign_arn,
        userId = str(user_id),
        filterArn = filter_arn
    )
    # Build a new dataframe of recommendations
    item_list = get_recommendations_response['itemList']
    recommendation_list = []
    for item in item_list:
        movie = get_movie_by_id(item['itemId'])
        recommendation_list.append(movie)
    #print(recommendation_list)
    filter_name = filter_arn.split('/')[1]
    new_rec_DF = pd.DataFrame(recommendation_list, columns = [filter_name])
    # Add this dataframe to the old one
    recommendations_df = pd.concat([recommendations_df, new_rec_DF], axis=1)
    return recommendations_df

Vous pouvez accéder aux recommandations de films dans un genre donné. Dans une application VOD, vous pouvez créer des répertoires (également appelés rails ou carrousels) facilement en utilisant ces filtres. En fonction des informations dont vous disposez sur vos articles, vous pouvez également filtrer selon des informations supplémentaires telles que le mot-clé, l'année/décennie, etc.

In [ ]:
recommendations_df_shelves = pd.DataFrame()
for filter_arn in meta_filter_arns:
    recommendations_df_shelves = get_new_recommendations_df_by_filter(recommendations_df_shelves, user, filter_arn)
for filter_arn in decade_filter_arns:
    recommendations_df_shelves = get_new_recommendations_df_by_filter(recommendations_df_shelves, user, filter_arn)

recommendations_df_shelves

La prochaine rubrique concerne les événements en temps réel. Personalize a la capacité de suivre les événements de votre application afin de mettre à jour les recommandations présentées à l'utilisateur. Cela est particulièrement utile pour les charges de travail liées aux médias, comme la vidéo à la demande, où l'intention du client peut différer selon qu'il regarde le film avec ses enfants ou seul.

En outre, les événements enregistrés par l’intermédiaire de ce système sont stockés jusqu'à ce que vous émettiez un ordre de suppression, et ils sont utilisés comme données historiques avec les autres données d'interaction que vous avez fournies lors de la formation de vos prochains modèles.

#### Événements en temps réel

Commencez par créer un suivi d'événement qui est attaché à la campagne.

In [ ]:
response = personalize.create_event_tracker(
    name='MovieTracker',
    datasetGroupArn=dataset_group_arn
)
print(response['eventTrackerArn'])
print(response['trackingId'])
TRACKING_ID = response['trackingId']
event_tracker_arn = response['eventTrackerArn']

Nous allons créer un code qui simule l'interaction d'un utilisateur avec un article particulier. Après avoir exécuté ce code, vous obtiendrez des recommandations qui diffèrent des résultats ci-dessus.

Nous commençons par créer quelques méthodes pour la simulation d'événements en temps réel.

In [ ]:
session_dict = {}

def send_movie_click(USER_ID, ITEM_ID, EVENT_TYPE):
    """
    Simulates a click as an envent
    to send an event to Amazon Personalize's Event Tracker
    """
    # Configure Session
    try:
        session_ID = session_dict[str(USER_ID)]
    except:
        session_dict[str(USER_ID)] = str(uuid.uuid1())
        session_ID = session_dict[str(USER_ID)]
        
    # Configure Properties:
    event = {
    "itemId": str(ITEM_ID),
    }
    event_json = json.dumps(event)
        
    # Make Call
    
    personalize_events.put_events(
    trackingId = TRACKING_ID,
    userId= str(USER_ID),
    sessionId = session_ID,
    eventList = [{
        'sentAt': int(time.time()),
        'eventType': str(EVENT_TYPE),
        'properties': event_json
        }]
    )

def get_new_recommendations_df_users_real_time(recommendations_df, user_id, item_id, event_type):
    # Get the artist name (header of column)
    movie_name = get_movie_by_id(item_id)
    # Interact with different movies
    print('sending event ' + event_type + ' for ' + get_movie_by_id(item_id))
    send_movie_click(USER_ID=user_id, ITEM_ID=item_id, EVENT_TYPE=event_type)
    # Get the recommendations (note you should have a base recommendation DF created before)
    get_recommendations_response = personalize_runtime.get_recommendations(
        campaignArn = userpersonalization_campaign_arn,
        userId = str(user_id),
    )
    # Build a new dataframe of recommendations
    item_list = get_recommendations_response['itemList']
    recommendation_list = []
    for item in item_list:
        artist = get_movie_by_id(item['itemId'])
        recommendation_list.append(artist)
    new_rec_DF = pd.DataFrame(recommendation_list, columns = [movie_name])
    # Add this dataframe to the old one
    #recommendations_df = recommendations_df.join(new_rec_DF)
    recommendations_df = pd.concat([recommendations_df, new_rec_DF], axis=1)
    return recommendations_df

À ce stade, nous n'avons pas encore généré d'événements en temps réel ; nous avons seulement configuré le code. Pour comparer les recommandations formulées avant et après les événements en temps réel, nous allons choisir un utilisateur et générer les recommandations originales pour lui.

In [ ]:
# First pick a user
user_id = user

# Get recommendations for the user
get_recommendations_response = personalize_runtime.get_recommendations(
        campaignArn = userpersonalization_campaign_arn,
        userId = str(user_id),
    )

# Build a new dataframe for the recommendations
item_list = get_recommendations_response['itemList']
recommendation_list = []
for item in item_list:
    artist = get_movie_by_id(item['itemId'])
    recommendation_list.append(artist)
user_recommendations_df = pd.DataFrame(recommendation_list, columns = [user_id])
user_recommendations_df

Nous disposons à présent d'une liste de recommandations pour cet utilisateur avant d'avoir appliqué des événements en temps réel. Choisissons à présent 3 artistes au hasard avec lesquels nous allons simuler l'interaction de notre utilisateur et, voyons comment cela modifie les recommandations.

In [ ]:
# Next generate 3 random movies
movies = items_df.sample(3).index.tolist()

In [ ]:
# Note this will take about 15 seconds to complete due to the sleeps
for movie in movies:
    user_recommendations_df = get_new_recommendations_df_users_real_time(user_recommendations_df, user_id, movie,'click')
    time.sleep(5)

Nous pouvons à présent examiner comment les événements de type clic ont modifié les recommandations.

In [ ]:
user_recommendations_df

Dans la cellule ci-dessus, la première colonne après l'index est constituée des recommandations par défaut de l'utilisateur issues de la personnalisation de l'utilisateur, et chaque colonne suivante comporte un en-tête de l'artiste avec lequel il a interagi par le biais d'un événement en temps réel ainsi que les recommandations à la suite de cet événement. 

Il est possible que le comportement ne change pas beaucoup. Cela est dû à la nature relativement limitée de ce jeu de données et à l'effet de quelques clics aléatoires. Pour mieux comprendre ce phénomène, essayez de simuler le fait de cliquer sur plusieurs films et vous devriez constater un impact notable.

Examinons à présent les filtres d'articles qui vous permettent de filtrer les articles en fonction des données d'interaction. Pour ce jeu de données, il pourrait s'agir d'un clic ou d'un visionnage en fonction des données que nous avons importées. Il pourrait aussi s'agir de n'importe quel schéma d'interaction que vous concevez (clic, taux, appréciation, lecture, achat, etc.). Pour les répertoires de VOD, vous pouvez faire passer un titre de la catégorie « Bons plans pour vous » à la catégorie « À revoir ». Les recommandations « À revoir » seront basées sur les interactions actuelles de l'utilisateur, mais ne recommanderont que les titres qui ont déjà été regardés.


In [ ]:
recommendations_df_events = pd.DataFrame()
for filter_arn in interaction_filter_arns:
    recommendations_df_events = get_new_recommendations_df_by_filter(recommendations_df_events, user, filter_arn)
    
recommendations_df_events

Nous allons à présent soumettre un événement de visionnage pour 4 recommandations non visionnées, ce qui simulerait le visionnage de 4 films. Dans une application de VOD, vous pouvez choisir de soumettre un événement après que l'utilisateur a regardé une bonne partie (plus de 75 %) du contenu. Si vous soumettez l'évènement lorsque l'utilisateur a regardé 100 % du contenu, les utilisateurs à court de crédit avant 100 % n'auront pas accès à l'évènement.

In [ ]:
 # Get the recommendations
top_unwatched_recommendations_response = personalize_runtime.get_recommendations(
    campaignArn = userpersonalization_campaign_arn,
    userId = str(user_id),
    filterArn = filter_arn,
    numResults=4)
item_list = top_unwatched_recommendations_response['itemList']
for item in item_list:
    print('sending event watch for ' + get_movie_by_id(item['itemId']))
    send_movie_click(USER_ID=user_id, ITEM_ID=item['itemId'], EVENT_TYPE='watch')
    time.sleep(10)

Nous pouvons à présent consulter les filtres d'événements pour voir les recommandations mises à jour, qu'elles soient suivies ou non. 

In [ ]:
recommendations_df_events = pd.DataFrame()
for filter_arn in interaction_filter_arns:
    recommendations_df_events = get_new_recommendations_df_by_filter(recommendations_df_events, user, filter_arn)
    
recommendations_df_events

### Classement personnalisé

Le cas d'utilisation principal du classement personnalisé consiste à traiter une collection d'articles et à les présenter par ordre de priorité ou d'intérêt probable pour un utilisateur. Pour une application de VOD, vous souhaitez présenter de manière dynamique un répertoire personnalisé (répertoire, rail, carrousel) en fonction de certaines informations (réalisateur, lieu, franchise de super-héros, période du film, etc.). Il se peut que ces informations ne figurent pas dans vos métadonnées, ce qui signifie qu'un filtre de métadonnées des articles ne fonctionnera pas. Cependant, vous pouvez disposer de ces informations dans votre système pour générer la liste des articles. 

Pour le démontrer, nous allons utiliser le même utilisateur que précédemment et une collection aléatoire d'articles.

In [ ]:
rerank_user = user
rerank_items = items_df.sample(25).index.tolist()

Créez maintenant un tableau de données qui montre les données d'entrée.

In [ ]:
rerank_list = []
for item in rerank_items:
    movie = get_movie_by_id(item)
    rerank_list.append(movie)
rerank_df = pd.DataFrame(rerank_list, columns = ['Un-Ranked'])
rerank_df

Lancez ensuite l'appel d'API de classement personnalisé.

In [ ]:
# Convert user to string:
user_id = str(rerank_user)
rerank_item_list = []
for item in rerank_items:
    rerank_item_list.append(str(item))
    
# Get recommended reranking
get_recommendations_response_rerank = personalize_runtime.get_personalized_ranking(
        campaignArn = rerank_campaign_arn,
        userId = user_id,
        inputList = rerank_item_list
)

Ajoutez à présent les articles reclassés dans une deuxième colonne du tableau de données original, pour une comparaison parallèle.

In [ ]:
ranked_list = []
item_list = get_recommendations_response_rerank['personalizedRanking']
for item in item_list:
    movie = get_movie_by_id(item['itemId'])
    ranked_list.append(movie)
ranked_df = pd.DataFrame(ranked_list, columns = ['Re-Ranked'])
rerank_df = pd.concat([rerank_df, ranked_df], axis=1)
rerank_df

Vous pouvez constater ci-dessus comment chaque entrée a été réorganisée en fonction de la compréhension du modèle par l'utilisateur. C'est une tâche courante lorsque vous avez une collection d'articles à présenter à un utilisateur, une liste de promotions par exemple.

## Recommandations par lots<a class="anchor" id="batch"></a>
[Retour au début](#top)

Dans de nombreux cas, vous pouvez souhaiter disposer d'un plus grand jeu de données de recommandations exportées. Récemment, Amazon Personalize a introduit les recommandations par lots comme moyen d'exporter une collection de recommandations vers S3. Dans cet exemple, nous allons étudier la manière de procéder pour la solution HRNN. Pour en savoir plus sur les recommandations par lots, veuillez vous reporter à la [documentation](https://docs.aws.amazon.com/personalize/latest/dg/getting-recommendations.html#recommendations-batch). Cette fonction s'applique à toutes les recettes, mais le format de sortie varie.

Une implémentation simple se présente comme suit :

```python
import boto3

personalize_rec = boto3.client(service_name='personalize')

personalize_rec.create_batch_inference_job (
    solutionVersionArn = "ARN de la version de la solution,"
    jobName = "Nom de la tâche du lot",
    roleArn = "ARN du rôle IAM",
    jobInput = 
       {"s3DataSource": {"path": S3 input path}},
    jobOutput = 
       {"s3DataDestination": {"path":S3 output path"}}
)
```

L'importation du kit SDK, l'ARN de la version de la solution et les ARN de rôle ont tous été définis. Il ne reste plus qu'à définir une entrée, une sortie et un nom de tâche.

Commençant par l'entrée pour HRNN, qui se présente comme suit :


```JSON
{"userId": "4638"}
{"userId": "663"}
{"userId": "3384"}
```

Le résultat devrait ressembler à ce qui suit :

```JSON
{"input":{"userId":"4638"}, "output": {"recommendedItems": ["296", "1", "260", "318"]}}
{"input":{"userId":"663"}, "output": {"recommendedItems": ["1393", "3793", "2701", "3826"]}}
{"input":{"userId":"3384"}, "output": {"recommendedItems": ["8368", "5989", "40815", "48780"]}}
```

La sortie est un fichier JSON Lines. Il est constitué d'objets JSON individuels, un par ligne. Nous devrons donc fournir un effort supplémentaire par la suite pour digérer les résultats dans ce format.

### Création du fichier d'entrée

Lorsque vous utilisez la fonction de traitement par lots, vous indiquez les utilisateurs pour lesquels vous souhaitez recevoir des recommandations lorsque la tâche est terminée. La cellule ci-dessous va à nouveau sélectionner quelques utilisateurs au hasard, puis créer le fichier et le sauvegarder sur le disque. À partir de là, vous le chargerez sur S3 pour l'utiliser plus tard dans l'appel d'API.

In [ ]:
# We will use the same users from before
users
# Write the file to disk
json_input_filename = "json_input.json"
with open(data_dir + "/" + json_input_filename, 'w') as json_input:
    for user_id in users:
        json_input.write('{"userId": "' + str(user_id) + '"}\n')

In [ ]:
# Showcase the input file:
!cat $data_dir"/"$json_input_filename

Chargez le fichier vers S3 et enregistrez le chemin d'accès comme variable pour plus tard.

In [ ]:
# Upload files to S3
boto3.Session().resource('s3').Bucket(bucket_name).Object(json_input_filename).upload_file(data_dir+"/"+json_input_filename)
s3_input_path = "s3://" + bucket_name + "/" + json_input_filename
print(s3_input_path)

Les recommandations par lots lisent l'entrée du fichier que nous avons chargé sur S3. De la même manière, les recommandations par lots enregistreront le résultat dans un fichier dans S3. Nous définissons donc le chemin de sortie où les résultats doivent être enregistrés.

In [ ]:
# Define the output path
s3_output_path = "s3://" + bucket_name + "/"
print(s3_output_path)

Maintenant, exécutez simplement l'appel pour lancer l'exportation par lots.

In [ ]:
batchInferenceJobArn = personalize.create_batch_inference_job (
    solutionVersionArn = userpersonalization_solution_version_arn,
    jobName = "VOD-POC-Batch-Inference-Job-UserPersonalization_" + str(round(time.time()*1000)),
    roleArn = role_arn,
    jobInput = 
     {"s3DataSource": {"path": s3_input_path}},
    jobOutput = 
     {"s3DataDestination":{"path": s3_output_path}}
)
batchInferenceJobArn = batchInferenceJobArn['batchInferenceJobArn']

Exécutez la boucle while ci-dessous pour suivre le statut de l'appel de recommandation par lot. Cette opération peut prendre environ 30 minutes, car Personalize doit mettre en place l'infrastructure pour effectuer cette tâche. Nous testons la fonction avec un jeu de données de seulement 3 utilisateurs, ce qui n'est pas une utilisation efficace de ce mécanisme. Normalement, vous n'utiliserez cette fonction que pour le traitement en masse, auquel cas les gains d'efficacité deviendront évidents.

In [ ]:
current_time = datetime.now()
print("Import Started on: ", current_time.strftime("%I:%M:%S %p"))

max_time = time.time() + 6*60*60 # 6 hours
while time.time() < max_time:
    describe_dataset_inference_job_response = personalize.describe_batch_inference_job(
        batchInferenceJobArn = batchInferenceJobArn
    )
    status = describe_dataset_inference_job_response["batchInferenceJob"]['status']
    print("DatasetInferenceJob: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)
    
current_time = datetime.now()
print("Import Completed on: ", current_time.strftime("%I:%M:%S %p"))

In [ ]:
s3 = boto3.client('s3')
export_name = json_input_filename + ".out"
s3.download_file(bucket_name, export_name, data_dir+"/"+export_name)

# Update DF rendering
pd.set_option('display.max_rows', 30)
with open(data_dir+"/"+export_name) as json_file:
    # Get the first line and parse it
    line = json.loads(json_file.readline())
    # Do the same for the other lines
    while line:
        # extract the user ID 
        col_header = "User: " + line['input']['userId']
        # Create a list for all the artists
        recommendation_list = []
        # Add all the entries
        for item in line['output']['recommendedItems']:
            movie = get_movie_by_id(item)
            recommendation_list.append(movie)
        if 'bulk_recommendations_df' in locals():
            new_rec_DF = pd.DataFrame(recommendation_list, columns = [col_header])
            bulk_recommendations_df = bulk_recommendations_df.join(new_rec_DF)
        else:
            bulk_recommendations_df = pd.DataFrame(recommendation_list, columns=[col_header])
        try:
            line = json.loads(json_file.readline())
        except:
            line = None
bulk_recommendations_df

## Récapitulatif <a class="anchor" id="wrapup"></a>
[Retour au début](#top)

Vous disposez désormais d'un ensemble de modèles parfaitement opérationnels pour aborder divers scénarios de recommandation et de personnalisation, ainsi que des compétences nécessaires pour manipuler les données des clients afin de mieux les intégrer au service. Vous saurez comment procéder par le biais de l'API et en exploitant des outils de science des données open source.

Utilisez ces bloc-notes comme guide pour démarrer avec vos clients pour les POC. Si vous trouvez des composants manquants ou si vous découvrez de nouvelles approches, faites une demande d'extraction et fournissez tout composant supplémentaire utile qui pourrait manquer dans cette collection.

Vous devrez vous assurer de supprimer toutes les ressources déployées dans le cadre de cette POC. Nous avons fourni un bloc-notes distinct qui vous montre comment identifier et supprimer les ressources dans `06_Clean_Up_Resources.ipynb`.

In [ ]:
%store event_tracker_arn
%store batchInferenceJobArn